In [1]:
import pandas as pd 
import os
import random

In [2]:
repeat0 = 10
repeat1 = 1000

In [3]:
code = r"""

; ------------------------------------
; Author: Hosein Yavarzadeh 
; Email: hyavarzadeh@ucsd.edu
; Supervisor: Prof. Dean Tullsen
; ------------------------------------

%define repeat0  {}             ; Number of repetitions of whole test. Default = 8
%define repeat1  {}           ; Repeat count for loop around testcode. Default = no loop
%define repeat2  1              ; Repeat count for repeat macro around testcode. Default = 100

%macro testinitc 0              ; Macro to call in each test before reading counters.
    mov dword[PrintCustomPMC], 1
    mov ebx, 8000             
    lea rdi, [UserData]
    rand_loop:
    rdrand eax
    and eax, 1
    mov [rdi], al
    inc rdi
    dec ebx
    jnz rand_loop

%endmacro

%macro testinit2 0              ; Macro with initializations before each test. Default = nothing
    lea rdi, [UserData]
%endmacro

%macro testcode 0               ; A multi-line macro executing any piece of test code.

    inc rdi

    cmp byte[rdi],0
    
    ; dummy_branch 300, 0

    ; jmp rand_branch
    mov rax, 0x100000000
    mov rbx, rand_branch_back
    mov rdx, {}
    mov rcx, 0
    cmp rcx, 0
    jmp rax
    rand_branch_back:

    dummy_branch var, 5

    ; test branch
    READ_PMC_START
    cmp byte[rdi],0
    je target2
    target2:
    READ_PMC_END

%endmacro

%macro dummy_branch 2
      mov rax, 0
      %assign i (100)*(%2)
      %assign j (100)*(%2)
      %rep %1
            %assign j i+1
            label%+ i:
            cmp rax, 0
            je label%+ j 
            %assign i j  
      %endrep
    ;   align (1 << my_align)
      label%+ j:
%endmacro

"""

In [4]:
ld_script = r"""
/* Script for -z combreloc -z separate-code */
/* Copyright (C) 2014-2020 Free Software Foundation, Inc.
   Copying and distribution of this script, with or without modification,
   are permitted in any medium without royalty provided the copyright
   notice and this notice are preserved.  */
OUTPUT_FORMAT("elf64-x86-64", "elf64-x86-64",
	      "elf64-x86-64")
OUTPUT_ARCH(i386:x86-64)
ENTRY(_start)
SEARCH_DIR("=/usr/local/lib/x86_64-linux-gnu"); SEARCH_DIR("=/lib/x86_64-linux-gnu"); SEARCH_DIR("=/usr/lib/x86_64-linux-gnu"); SEARCH_DIR("=/usr/lib/x86_64-linux-gnu64"); SEARCH_DIR("=/usr/local/lib64"); SEARCH_DIR("=/lib64"); SEARCH_DIR("=/usr/lib64"); SEARCH_DIR("=/usr/local/lib"); SEARCH_DIR("=/lib"); SEARCH_DIR("=/usr/lib"); SEARCH_DIR("=/usr/x86_64-linux-gnu/lib64"); SEARCH_DIR("=/usr/x86_64-linux-gnu/lib");
SECTIONS
{
  PROVIDE (__executable_start = SEGMENT_START("text-segment", 0x400000)); . = SEGMENT_START("text-segment", 0x400000) + SIZEOF_HEADERS;
  .interp         : { *(.interp) }
  .note.gnu.build-id  : { *(.note.gnu.build-id) }
  .hash           : { *(.hash) }
  .gnu.hash       : { *(.gnu.hash) }
  .dynsym         : { *(.dynsym) }
  .dynstr         : { *(.dynstr) }
  .gnu.version    : { *(.gnu.version) }
  .gnu.version_d  : { *(.gnu.version_d) }
  .gnu.version_r  : { *(.gnu.version_r) }
  .rela.dyn       :
    {
      *(.rela.init)
      *(.rela.text .rela.text.* .rela.gnu.linkonce.t.*)
      *(.rela.fini)
      *(.rela.rodata .rela.rodata.* .rela.gnu.linkonce.r.*)
      *(.rela.data .rela.data.* .rela.gnu.linkonce.d.*)
      *(.rela.tdata .rela.tdata.* .rela.gnu.linkonce.td.*)
      *(.rela.tbss .rela.tbss.* .rela.gnu.linkonce.tb.*)
      *(.rela.ctors)
      *(.rela.dtors)
      *(.rela.got)
      *(.rela.bss .rela.bss.* .rela.gnu.linkonce.b.*)
      *(.rela.ldata .rela.ldata.* .rela.gnu.linkonce.l.*)
      *(.rela.lbss .rela.lbss.* .rela.gnu.linkonce.lb.*)
      *(.rela.lrodata .rela.lrodata.* .rela.gnu.linkonce.lr.*)
      *(.rela.ifunc)
    }
  .rela.plt       :
    {
      *(.rela.plt)
      PROVIDE_HIDDEN (__rela_iplt_start = .);
      *(.rela.iplt)
      PROVIDE_HIDDEN (__rela_iplt_end = .);
    }
  . = ALIGN(CONSTANT (MAXPAGESIZE));
  .init           :
  {
    KEEP (*(SORT_NONE(.init)))
  }
  .plt            : { *(.plt) *(.iplt) }
.plt.got        : { *(.plt.got) }
.plt.sec        : { *(.plt.sec) }
  . = 0x40000000;
  .text           :
  {
    b64.o (.text .stub .text.* .gnu.linkonce.t.*)
    *(.text.unlikely .text.*_unlikely .text.unlikely.*)
    *(.text.exit .text.exit.*)
    *(.text.startup .text.startup.*)
    *(.text.hot .text.hot.*)
    *(SORT(.text.sorted.*))
    a64.o (.text .stub .text.* .gnu.linkonce.t.*)
    /* .gnu.warning sections are handled specially by elf.em.  */
    *(.gnu.warning)
  }
  .fini           :
  {
    KEEP (*(SORT_NONE(.fini)))
  }
  PROVIDE (__etext = .);
  PROVIDE (_etext = .);
  PROVIDE (etext = .);
  . = ALIGN(CONSTANT (MAXPAGESIZE));
  /* Adjust the address for the rodata segment.  We want to adjust up to
     the same address within the page on the next page up.  */
  . = SEGMENT_START("rodata-segment", ALIGN(CONSTANT (MAXPAGESIZE)) + (. & (CONSTANT (MAXPAGESIZE) - 1)));
  .rodata         : { *(.rodata .rodata.* .gnu.linkonce.r.*) }
  .rodata1        : { *(.rodata1) }
  .eh_frame_hdr   : { *(.eh_frame_hdr) *(.eh_frame_entry .eh_frame_entry.*) }
  .eh_frame       : ONLY_IF_RO { KEEP (*(.eh_frame)) *(.eh_frame.*) }
  .gcc_except_table   : ONLY_IF_RO { *(.gcc_except_table .gcc_except_table.*) }
  .gnu_extab   : ONLY_IF_RO { *(.gnu_extab*) }
  /* These sections are generated by the Sun/Oracle C++ compiler.  */
  .exception_ranges   : ONLY_IF_RO { *(.exception_ranges*) }
  /* Adjust the address for the data segment.  We want to adjust up to
     the same address within the page on the next page up.  */
  . = DATA_SEGMENT_ALIGN (CONSTANT (MAXPAGESIZE), CONSTANT (COMMONPAGESIZE));
  /* Exception handling  */
  .eh_frame       : ONLY_IF_RW { KEEP (*(.eh_frame)) *(.eh_frame.*) }
  .gnu_extab      : ONLY_IF_RW { *(.gnu_extab) }
  .gcc_except_table   : ONLY_IF_RW { *(.gcc_except_table .gcc_except_table.*) }
  .exception_ranges   : ONLY_IF_RW { *(.exception_ranges*) }
  /* Thread Local Storage sections  */
  .tdata	  :
   {
     PROVIDE_HIDDEN (__tdata_start = .);
     *(.tdata .tdata.* .gnu.linkonce.td.*)
   }
  .tbss		  : { *(.tbss .tbss.* .gnu.linkonce.tb.*) *(.tcommon) }
  .preinit_array    :
  {
    PROVIDE_HIDDEN (__preinit_array_start = .);
    KEEP (*(.preinit_array))
    PROVIDE_HIDDEN (__preinit_array_end = .);
  }
  .init_array    :
  {
    PROVIDE_HIDDEN (__init_array_start = .);
    KEEP (*(SORT_BY_INIT_PRIORITY(.init_array.*) SORT_BY_INIT_PRIORITY(.ctors.*)))
    KEEP (*(.init_array EXCLUDE_FILE (*crtbegin.o *crtbegin?.o *crtend.o *crtend?.o ) .ctors))
    PROVIDE_HIDDEN (__init_array_end = .);
  }
  .fini_array    :
  {
    PROVIDE_HIDDEN (__fini_array_start = .);
    KEEP (*(SORT_BY_INIT_PRIORITY(.fini_array.*) SORT_BY_INIT_PRIORITY(.dtors.*)))
    KEEP (*(.fini_array EXCLUDE_FILE (*crtbegin.o *crtbegin?.o *crtend.o *crtend?.o ) .dtors))
    PROVIDE_HIDDEN (__fini_array_end = .);
  }
  .ctors          :
  {
    /* gcc uses crtbegin.o to find the start of
       the constructors, so we make sure it is
       first.  Because this is a wildcard, it
       doesn't matter if the user does not
       actually link against crtbegin.o; the
       linker won't look for a file to match a
       wildcard.  The wildcard also means that it
       doesn't matter which directory crtbegin.o
       is in.  */
    KEEP (*crtbegin.o(.ctors))
    KEEP (*crtbegin?.o(.ctors))
    /* We don't want to include the .ctor section from
       the crtend.o file until after the sorted ctors.
       The .ctor section from the crtend file contains the
       end of ctors marker and it must be last */
    KEEP (*(EXCLUDE_FILE (*crtend.o *crtend?.o ) .ctors))
    KEEP (*(SORT(.ctors.*)))
    KEEP (*(.ctors))
  }
  .dtors          :
  {
    KEEP (*crtbegin.o(.dtors))
    KEEP (*crtbegin?.o(.dtors))
    KEEP (*(EXCLUDE_FILE (*crtend.o *crtend?.o ) .dtors))
    KEEP (*(SORT(.dtors.*)))
    KEEP (*(.dtors))
  }
  .jcr            : { KEEP (*(.jcr)) }
  .data.rel.ro : { *(.data.rel.ro.local* .gnu.linkonce.d.rel.ro.local.*) *(.data.rel.ro .data.rel.ro.* .gnu.linkonce.d.rel.ro.*) }
  .dynamic        : { *(.dynamic) }
  .got            : { *(.got) *(.igot) }
  . = DATA_SEGMENT_RELRO_END (SIZEOF (.got.plt) >= 24 ? 24 : 0, .);
  .got.plt        : { *(.got.plt) *(.igot.plt) }
  .data           :
  {
    *(.data .data.* .gnu.linkonce.d.*)
    SORT(CONSTRUCTORS)
  }
  .data1          : { *(.data1) }
  _edata = .; PROVIDE (edata = .);
  . = .;
  __bss_start = .;
  .bss            :
  {
   *(.dynbss)
   *(.bss .bss.* .gnu.linkonce.b.*)
   *(COMMON)
   /* Align here to ensure that the .bss section occupies space up to
      _end.  Align after .bss to ensure correct alignment even if the
      .bss section disappears because there are no input sections.
      FIXME: Why do we need it? When there is no .bss section, we do not
      pad the .data section.  */
   . = ALIGN(. != 0 ? 64 / 8 : 1);
  }
  .lbss   :
  {
    *(.dynlbss)
    *(.lbss .lbss.* .gnu.linkonce.lb.*)
    *(LARGE_COMMON)
  }
  . = ALIGN(64 / 8);
  . = SEGMENT_START("ldata-segment", .);
  .lrodata   ALIGN(CONSTANT (MAXPAGESIZE)) + (. & (CONSTANT (MAXPAGESIZE) - 1)) :
  {
    *(.lrodata .lrodata.* .gnu.linkonce.lr.*)
  }
  .ldata   ALIGN(CONSTANT (MAXPAGESIZE)) + (. & (CONSTANT (MAXPAGESIZE) - 1)) :
  {
    *(.ldata .ldata.* .gnu.linkonce.l.*)
    . = ALIGN(. != 0 ? 64 / 8 : 1);
  }
  . = ALIGN(64 / 8);
  _end = .; PROVIDE (end = .);
  . = DATA_SEGMENT_END (.);
  /* Stabs debugging sections.  */
  .stab          0 : { *(.stab) }
  .stabstr       0 : { *(.stabstr) }
  .stab.excl     0 : { *(.stab.excl) }
  .stab.exclstr  0 : { *(.stab.exclstr) }
  .stab.index    0 : { *(.stab.index) }
  .stab.indexstr 0 : { *(.stab.indexstr) }
  .comment       0 : { *(.comment) }
  .gnu.build.attributes : { *(.gnu.build.attributes .gnu.build.attributes.*) }
  /* DWARF debug sections.
     Symbols in the DWARF debugging sections are relative to the beginning
     of the section so we begin them at 0.  */
  /* DWARF 1 */
  .debug          0 : { *(.debug) }
  .line           0 : { *(.line) }
  /* GNU DWARF 1 extensions */
  .debug_srcinfo  0 : { *(.debug_srcinfo) }
  .debug_sfnames  0 : { *(.debug_sfnames) }
  /* DWARF 1.1 and DWARF 2 */
  .debug_aranges  0 : { *(.debug_aranges) }
  .debug_pubnames 0 : { *(.debug_pubnames) }
  /* DWARF 2 */
  .debug_info     0 : { *(.debug_info .gnu.linkonce.wi.*) }
  .debug_abbrev   0 : { *(.debug_abbrev) }
  .debug_line     0 : { *(.debug_line .debug_line.* .debug_line_end) }
  .debug_frame    0 : { *(.debug_frame) }
  .debug_str      0 : { *(.debug_str) }
  .debug_loc      0 : { *(.debug_loc) }
  .debug_macinfo  0 : { *(.debug_macinfo) }
  /* SGI/MIPS DWARF 2 extensions */
  .debug_weaknames 0 : { *(.debug_weaknames) }
  .debug_funcnames 0 : { *(.debug_funcnames) }
  .debug_typenames 0 : { *(.debug_typenames) }
  .debug_varnames  0 : { *(.debug_varnames) }
  /* DWARF 3 */
  .debug_pubtypes 0 : { *(.debug_pubtypes) }
  .debug_ranges   0 : { *(.debug_ranges) }
  /* DWARF Extension.  */
  .debug_macro    0 : { *(.debug_macro) }
  .debug_addr     0 : { *(.debug_addr) }
  .gnu.attributes 0 : { KEEP (*(.gnu.attributes)) }
  /DISCARD/ : { *(.note.GNU-stack) *(.gnu_debuglink) *(.gnu.lto_*) }

"""

In [5]:
def create_nasm(cnt, rep):
    nasm_new_sections = ''
    for i in range(cnt):
        nasm_new_sections += f"""
SECTION .dummy_branch_{i} exec
    dummy_label_{i}:
    je dummy_label_{i+1}

"""
    nasm_new_sections += f"""
SECTION .dummy_branch_{cnt} exec
    dummy_label_{cnt}:
    dec rdx
    cmp rdx, 0
    jmp dummy_label_{cnt+1}

SECTION .dummy_branch_{cnt+1} exec
    dummy_label_{cnt+1}:
    je dummy_label_{cnt+2}
    mov rax, 0x100000000
    mov rcx, 0
    cmp rcx, 0
    jmp rax

SECTION .dummy_branch_{cnt+2} exec
    dummy_label_{cnt+2}:
    cmp byte[rdi],0
    jmp dummy_label_{cnt+3}

SECTION .dummy_branch_{cnt+3} exec
    dummy_label_{cnt+3}:
    je dummy_label_{cnt+4}
    jmp dummy_label_{cnt+4}

SECTION .dummy_branch_{cnt+4} exec
    dummy_label_{cnt+4}:
    jmp rbx
"""
    with open("bp.nasm", "w") as file:
      file.write(code.format(repeat0, repeat1, rep) + nasm_new_sections)

In [6]:
def create_ld(a, b, cnt, stride=0x10000000, offset=0x100000000, flip_mask=0x0, crit_bits=0x0):
  ld_new_sections = ''

  for i in range(cnt):
    ld_new_sections += f"""
    . = {hex(offset + i * stride)};
    .dummy_branch_{i} : {{ *(.dummy_branch_{i}) }}
  """

  # ld_new_sections += f"""
  #   . = {hex(0x100000000 + cnt * stride + ((1 << a) if a >= 0 else 0))};
  #   .dummy_branch_{cnt} : {{ *(.dummy_branch_{cnt}) }}
  # """

  # ld_new_sections += f"""
  #   . = {hex(0x100000000 + (cnt + 1) * stride + ((b << b) if a >= 0 else 0))};
  #   .dummy_branch_{cnt + 1} : {{ *(.dummy_branch_{cnt + 1}) }}
  # """
  
  # b_addr = 0x100000000 + cnt * stride + random.randint(0, ((1 << a) if a >= 0 else 0)) & 0xffffffffdf # 0x20
  # b_addr = 0x100000000 + cnt * stride + 0x315a2fb ^ flip_mask
  b_addr = 0x100000000 + cnt * stride
  b_addr = b_addr | crit_bits
  # b_addr = b_addr ^ flip_mask
  # b_addr = 0x44315a2fb
  # b_addr = 0x440000038
  
  
  
  ld_new_sections += f"""
    . = {hex(b_addr)};
    .dummy_branch_{cnt} : {{ *(.dummy_branch_{cnt}) }}
  """
  
  t_addr = 0x100000000 + (cnt + 1) * stride + random.randint(0, ((1 << b) if b >= 0 else 0)) # 0x20
  # t_addr = 0x482f2fbe4

  ld_new_sections += f"""
    . = {hex(t_addr)};
    .dummy_branch_{cnt + 1} : {{ *(.dummy_branch_{cnt + 1}) }}
  """

  ld_new_sections += '}'

  with open("script.ld", "w") as file:
      file.write(ld_script + ld_new_sections)
      
  return b_addr, t_addr

In [7]:
# rdx 5
# rep = 400
cnt = 10
start = 10
end = 10

i = 26
j = 26

final_dfss = []
for rep in range(1, 21):
    final_dfs = []
    for l in range(16,25):
        print(rep, l)
        final_df = []
        for k in range(48,64):
            create_nasm(cnt, rep)
            b_addr, t_addr = create_ld(i, j, cnt + 3, 1 << l, 0x100000000, 0x0, k)
            os.system(f"./run.sh {start} {end}")
            # if sample_cnt % 50 == 0:
            #     print(sample_cnt)

            data = pd.read_csv("results.csv", comment='#', delim_whitespace=True)
            data["Clock"] = pd.to_numeric(data["Clock"],errors='coerce')
            data["BrRetired"] = pd.to_numeric(data["BrRetired"],errors='coerce')
            data["BrMispred"] = pd.to_numeric(data["BrMispred"],errors='coerce')
            data["Clock"] = (data["Clock"]) / repeat1
            data["BrRetired"] = data["BrRetired"] / repeat1
            data["BrMispred"] = data["BrMispred"] / repeat1
            final_df.append((data.groupby(data.index // repeat0).mean()["BrMispred"][0], b_addr, t_addr))
        final_dfs.append(final_df)
    final_dfss.append(final_dfs)

1 16
1 17
1 18
1 19
1 20
1 21
1 22
1 23
1 24
2 16
2 17
2 18
2 19
2 20
2 21
2 22
2 23
2 24
3 16
3 17
3 18
3 19
3 20
3 21
3 22
3 23
3 24
4 16
4 17
4 18
4 19
4 20
4 21
4 22
4 23
4 24
5 16
5 17
5 18
5 19
5 20
5 21
5 22
5 23
5 24
6 16
6 17
6 18
6 19
6 20
6 21
6 22
6 23
6 24
7 16
7 17
7 18
7 19
7 20
7 21
7 22
7 23
7 24
8 16
8 17
8 18
8 19
8 20
8 21
8 22
8 23
8 24
9 16
9 17
9 18
9 19
9 20
9 21
9 22
9 23
9 24
10 16
10 17
10 18
10 19
10 20
10 21
10 22
10 23
10 24
11 16
11 17
11 18
11 19
11 20
11 21
11 22
11 23
11 24
12 16
12 17
12 18
12 19
12 20
12 21
12 22
12 23
12 24
13 16
13 17
13 18
13 19
13 20
13 21
13 22
13 23
13 24
14 16
14 17
14 18
14 19
14 20
14 21
14 22
14 23
14 24
15 16
15 17
15 18
15 19
15 20
15 21
15 22
15 23
15 24
16 16
16 17
16 18
16 19
16 20
16 21
16 22
16 23
16 24
17 16
17 17
17 18
17 19
17 20
17 21
17 22
17 23
17 24
18 16
18 17
18 18
18 19
18 20
18 21
18 22
18 23
18 24
19 16
19 17
19 18
19 19
19 20
19 21
19 22
19 23
19 24
20 16
20 17
20 18
20 19
20 20
20 21
20 22
20 23
20 24


In [8]:
for dt, final_dfs in enumerate(final_dfss, 1):
    print(dt, ' ---------------------')
    for ct, df in enumerate(final_dfs, 16):
        print(ct, len([(m, b, t) for (m, b, t) in df if m < 0.25]))

1  ---------------------
16 16
17 16
18 16
19 16
20 16
21 16
22 16
23 16
24 16
2  ---------------------
16 16
17 16
18 16
19 16
20 16
21 16
22 16
23 16
24 16
3  ---------------------
16 16
17 16
18 16
19 16
20 16
21 16
22 16
23 16
24 16
4  ---------------------
16 16
17 16
18 16
19 16
20 16
21 16
22 16
23 16
24 16
5  ---------------------
16 16
17 16
18 16
19 16
20 10
21 10
22 11
23 16
24 16
6  ---------------------
16 16
17 16
18 16
19 16
20 12
21 12
22 10
23 16
24 16
7  ---------------------
16 16
17 16
18 16
19 16
20 7
21 12
22 9
23 16
24 16
8  ---------------------
16 16
17 16
18 16
19 16
20 8
21 12
22 10
23 16
24 16
9  ---------------------
16 16
17 16
18 16
19 16
20 9
21 9
22 12
23 16
24 16
10  ---------------------
16 16
17 16
18 16
19 16
20 9
21 10
22 9
23 16
24 16
11  ---------------------
16 16
17 16
18 16
19 16
20 5
21 5
22 5
23 16
24 16
12  ---------------------
16 16
17 16
18 16
19 16
20 5
21 5
22 5
23 16
24 16
13  ---------------------
16 16
17 16
18 16
19 16
20 5
21 5
22

In [9]:
for m, b, t in final_dfs[-1]:
    if m < 0.25:
        print((0, hex(b), hex(t)))
    # else:
    #     print((m, hex(b), hex(t)))

(0, '0x10d000030', '0x111af532b')
(0, '0x10d000031', '0x10f521a94')
(0, '0x10d000032', '0x10fa03eaa')
(0, '0x10d000033', '0x111178702')
(0, '0x10d000034', '0x10e8cf321')
(0, '0x10d000035', '0x1110c151d')
(0, '0x10d000036', '0x10fa92568')
(0, '0x10d000037', '0x11093818c')
(0, '0x10d000038', '0x111b4c61f')
(0, '0x10d000039', '0x10ed04589')
(0, '0x10d00003a', '0x110f022be')
(0, '0x10d00003b', '0x111dc15d1')
(0, '0x10d00003c', '0x10e8a3d6a')
(0, '0x10d00003d', '0x1109a676a')
(0, '0x10d00003e', '0x10f4f0323')
(0, '0x10d00003f', '0x1117f4ccf')


In [10]:
my_df = pd.DataFrame(final_df)

In [11]:
my_df

,0,1,2
0,0.0007,4513071152,4591670059
1,0.0006,4513071153,4552006292
2,0.0012,4513071154,4557127338
3,0.0014,4513071155,4581721858
4,0.0013,4513071156,4539085601
5,0.0011,4513071157,4580971805
6,0.0006,4513071158,4557710696
7,0.0015,4513071159,4573069708
8,0.0008,4513071160,4592027167
9,0.0011,4513071161,4543497609


In [12]:
hex(18305360635)

'0x44315a2fb'

In [13]:
# my_df = pd.DataFrame(final_dfs)
# my_df.to_pickle('./script11_4/1.pkl')